# Spanning Tree

In this notebook you will:

-   Reserve resources for the Spanning Tree experiment: a LAN with four hosts and four bridges
-   Configure your reserved resources
-   Access your reserved resources over SSH
-   Retrieve files saved on a FABRIC resources
-   Delete your FABRIC resources to free them for other experimenters

### Configure environment

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

### Define configuration for Spanning Tree experiment

In [ ]:
slice_name="stp-" + fablib.get_bastion_username()

node_conf = [
 {'name': "romeo",   'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': []}, 
 {'name': "hamlet",  'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': []}, 
 {'name': "othello",  'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': []}, 
 {'name': "petruchio", 'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': []}, 
 {'name': "bridge-1", 'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': []}, 
 {'name': "bridge-2", 'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': []}, 
 {'name': "bridge-3", 'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': []}, 
 {'name': "bridge-4", 'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_22', 'packages': []}
]
net_conf = [
 {"name": "net0", "subnet": "10.10.0.0/24", "nodes": [{"name": "hamlet", "addr": "10.10.0.102"}, {"name": "bridge-1", "addr": None}, {"name": "bridge-2", "addr": None}]},
 {"name": "net1", "subnet": "10.10.0.0/24", "nodes": [{"name": "othello", "addr": "10.10.0.104"}, {"name": "bridge-2", "addr": None}, {"name": "bridge-3", "addr": None}]},
 {"name": "net2", "subnet": "10.10.0.0/24", "nodes": [{"name": "petruchio", "addr": "10.10.0.106"}, {"name": "bridge-1", "addr": None}, {"name": "bridge-4", "addr": None}]},
 {"name": "net3", "subnet": "10.10.0.0/24", "nodes": [{"name": "romeo", "addr": "10.10.0.100"}, {"name": "bridge-3", "addr": None}, {"name": "bridge-4", "addr": None}]}
]
route_conf = []
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

### Reserve resources

Now, we are ready to reserve resources!

First, make sure you don’t already have a slice with this name:

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

We will select a random site that has sufficient resources for our experiment:

In [ ]:
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

fablib.show_site(site_name)

Then we will add hosts and network segments:

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

-   While it is being prepared, the “State” of the slice will appear as “Configuring”.
-   When it is ready, the “State” of the slice will change to “StableOK”.

You may prefer to walk away and come back in a few minutes (for simple slices) or a few tens of minutes (for more complicated slices with many resources).

In [ ]:
slice.submit()

In [ ]:
slice.get_state()
slice.wait_ssh(progress=True)

### Configure resources

Next, we will configure the resources so they are ready to use.

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

#### Draw the network topology

The following cell will draw the network topology, for your reference. The interface name and addresses of each experiment interface will be shown on the drawing.

In [ ]:
l2_nets = [(n.get_name(), {'color': 'lavender'}) for n in slice.get_l2networks() ]
l3_nets = [(n.get_name(), {'color': 'pink'}) for n in slice.get_l3networks() ]
hosts   =   [(n.get_name(), {'color': 'lightblue'}) for n in slice.get_nodes()]
nodes = l2_nets + l3_nets + hosts
ifaces = [iface.toDict() for iface in slice.get_interfaces()]
edges = [(iface['network'], iface['node'], 
          {'label': iface['physical_dev'] + '\n' + iface['ip_addr'] + '\n' + iface['mac']}) for iface in ifaces]

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
plt.figure(figsize=(len(nodes),len(nodes)))
G = nx.Graph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)
pos = nx.spring_layout(G)
nx.draw(G, pos, node_shape='s',  
        node_color=[n[1]['color'] for n in nodes], 
        node_size=[len(n[0])*400 for n in nodes],  
        with_labels=True);
nx.draw_networkx_edge_labels(G,pos,
                             edge_labels=nx.get_edge_attributes(G,'label'),
                             font_color='gray',  font_size=8, rotate=False);

### Log into resources

Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the resources in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
slice_info = [{'Name': n.get_name(), 'SSH command': n.get_ssh_command()} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the resources as follows:

-   in Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each resource. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.

### Retrieve files from Spanning Tree experiment

As you complete each part of the experiment, you may choose to transfer packet captures from the remote hosts to this Jupyter environment. Then, you can download them from the Jupyter environment to open in Wireshark.

To download a file from the Jupyter environment, use the file browser on the left side. You may need to use the “Refresh” button to see the updated file list after transferring a file. Then, you can right-click on a file and select “Download” to download it to your own computer.

In [ ]:
import os
romeo_exec = slice.get_node("romeo")
romeo_name = romeo_exec.execute("hostname", quiet=True)[0].strip()
hamlet_exec = slice.get_node("hamlet")
hamlet_name = hamlet_exec.execute("hostname", quiet=True)[0].strip()
othello_exec = slice.get_node("othello")
othello_name = othello_exec.execute("hostname", quiet=True)[0].strip()
petruchio_exec = slice.get_node("petruchio")
petruchio_name = petruchio_exec.execute("hostname", quiet=True)[0].strip()

host_vars = [
    ("romeo", romeo_exec, romeo_name),
    ("juliet", hamlet_exec, hamlet_name),
    ("othello", othello_exec, othello_name),
    ("petruchio", petruchio_exec, petruchio_name)
]

#### Packet Captures for Network of one bridge

In [ ]:
for node_name, host_exec, host_name in host_vars:
    host_1_pcap = "/home/ubuntu/stp-%s-1.pcap" % host_name
    host_exec.download_file(os.path.abspath('stp-%s-1.pcap' % node_name), host_1_pcap)

#### Packet Captures for Network of two bridges

In [ ]:
for node_name, host_exec, host_name in host_vars:
    host_2_pcap = "/home/ubuntu/stp-%s-2.pcap" % host_name
    host_exec.download_file(os.path.abspath('stp-%s-2.pcap' % node_name), host_2_pcap)

#### Packet Captures for Network of three bridges

In [ ]:
for node_name, host_exec, host_name in host_vars:
    host_3_pcap = "/home/ubuntu/stp-%s-3.pcap" % host_name
    host_exec.download_file(os.path.abspath('stp-%s-3.pcap' % node_name), host_3_pcap)

#### Packet Captures for Network of four bridges

In [ ]:
for node_name, host_exec, host_name in host_vars:
    host_4_pcap = "/home/ubuntu/stp-%s-4.pcap" % host_name
    host_exec.download_file(os.path.abspath('stp-%s-4.pcap' % node_name), host_4_pcap)

#### Packet Captures for Adapt to changes in the topology

In [ ]:
for node_name, host_exec, host_name in host_vars:
    host_change_pcap = "/home/ubuntu/stp-change-%s.pcap" %  host_name
    host_exec.download_file(os.path.abspath('stp-change-%s.pcap' % node_name), host_change_pcap)

### Delete your slice

When you finish your experiment, you should delete your slice! The following cells deletes all the resources in your slice, freeing them for other experimenters.

In [ ]:
slice = fablib.get_slice(name=slice_name)
fablib.delete_slice(slice_name)

In [ ]:
# slice should end up in "Dead" state
# re-run this cell until you see it in "Dead" state
slice.update()
_ = slice.show()